In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
results_folder = 'results'
os.environ['CITYSCAPES_RESULTS'] = results_folder
os.environ['CITYSCAPES_DATASET'] = 'data/gtFine_trainvaltest'


In [2]:
import numpy as np
from PIL import Image

from generator import CityScapesGenerator

import cityscapesScripts.cityscapesscripts.helpers.labels as labels


In [26]:
from model_residual_v2 import create_model
model = create_model((1024, 2048, 3), width=16, depth=15)
model.load_weights('model_residual_v2_16_15_50ep.h5')
# model.load_weights('model_residual_v2_16_15_50ep_myloss.h5')
# model.load_weights('model_residual_v2_16_15_50ep_mylossgamma2.h5')

In [22]:
from model_residual import create_model
model = create_model((1024, 2048, 3), width=64, depth=9)
model.load_weights('models/model_residual_64_9.h5')

In [23]:
def post_process(pred):
    trn_ids = np.argmax(pred, axis=-1)
    ids = np.vectorize(lambda x: labels.trainId2label[x].id)(trn_ids)
    return ids

def save_pred(pred, filename):
    colors = [lbl.color for lbl in labels.labels]
    colors = np.array(colors).astype('uint8')
    img = Image.fromarray(pred.astype('uint8'))
    img.putpalette(colors)
    img.save(filename)

In [27]:
!rm -r $results_folder
!mkdir $results_folder

gen = CityScapesGenerator('val', dir_='data', augment=False, shuffle=False)
for i in range(len(gen)):
    print(i, end='\r')
    path = gen.imgs[i]
    out_path = os.path.join(results_folder, path.split('/')[-1])
    pred = model.predict(gen[i])[0]
    pred = post_process(pred)
    save_pred(pred, out_path)


In [ ]:
# model_residual_v2, 16 width, 15 depth
# 1st one: 49.5 mIoU  (focal loss)
# 2nd one: 38.0 mIoU  (my loss)
# 3rd one: 34.5 mIoU  (my focal loss)

# model_residual, 64 width, 9 depth
# 49.2 mIoU  (focal loss)

In [28]:
import cityscapesScripts.cityscapesscripts.evaluation.evalPixelLevelSemanticLabeling as evalPixelLevelSemanticLabeling

# the main call isn't working for some reason...
if False:
    evalPixelLevelSemanticLabeling.main()
else:
    import glob
    args = evalPixelLevelSemanticLabeling.args
    predictionImgList = []
    groundTruthImgList = []
    groundTruthImgList = glob.glob(args.groundTruthSearch)
    for gt in groundTruthImgList:
        predictionImgList.append( evalPixelLevelSemanticLabeling.getPrediction(args,gt) )
    evalPixelLevelSemanticLabeling.evaluateImgLists(predictionImgList, groundTruthImgList, args)


Evaluating 500 pairs of images...
Images Processed: 500 

-------------- ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------- 
              |  u   |  e   |  r   |  o   |  s   |  d   |  g   |  r   |  s   |  p   |  r   |  b   |  w   |  f   |  g   |  b   |  t   |  p   |  p   |  t   |  t   |  v   |  t   |  s   |  p   |  r   |  c   |  t   |  b   |  c   |  t   |  t   |  m   |  b   | Prior |
-------------- ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------- 
    unlabeled | 0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.09   0.00   0.00   0.00   0.00   0.00   0.01   0.00   0.0